# 星宸药研 | DeepChem中文教程1.
# 使用DeepChem训练模型：一个简单的例子

DeepChem是一个专门用于化学、材料科学和生物学的机器学习库，支持多种数据类型和模型结构。
在本教程中，将展示如何使用DeepChem来训练一个简单的机器学习模型。


DeepChem GitHub仓库: [https://github.com/deepchem/deepchem](https://github.com/deepchem/deepchem)

星宸药研 GitHub仓库: [https://github.com/stardrug-c01/Deepchem-Chinese](https://github.com/stardrug-c01/Deepchem-Chinese)

本中文教程将保存于星宸药研 GitHub以及星宸药研微信公众号。

## 前情提要

在使用本教程之前，你需要先安装DeepChem的环境。请参考以下链接进行安装：
[如何安装DeepChem环境](https://mp.weixin.qq.com/s/dDyKw7LFTtFKJyyHt54xmw)

这是小编使用的conda环境概述:

## 关键依赖项 (Dependencies)
- **cudatoolkit**: `11.3`     
- **cudnn**: `8.9.2.26`
- **Python**: `3.9.19`
- **DeepChem**: `2.5.0`
- **TensorFlow**: `2.10.1` 
- **PyTorch**: `1.12.1` (CUDA 11.3)   #没有显卡可到pytorch官网找cpu版本的下载命令
- **TorchDrug**: `0.2.1`
- **RDKit**: `2024.03.5`
- **Keras**: `3.4.1`
- **NumPy**: `1.26.4`
- **Pandas**: `2.2.2`
- **Scikit-learn**: `1.5.1`
- **SciPy**: `1.13.1`
- **MATPLOTLIB**: `3.9.1`

## 其他依赖项 (Other Dependencies)

- **CFFI**: `1.16.0`
- **IPython**: `8.18.1`
- **JupyterLab**: `4.2.4`
- **Bokeh**: `3.3.2`
- **TQDM**: `4.66.4`
- **Tornado**: `6.4.1`
- **protobuf**: `4.25.4`
- **GRPCIO**: `1.65.1`
- **Tensorboard**: `2.17.0`

---


In [ ]:
# 确认Deepchem已安装

import deepchem as dc
dc.__version__

# 使用DeepChem训练模型：一个简单的例子

深度学习可以用来解决各种各样的问题，但基本的工作流程通常是相同的。以下是你需要遵循的典型步骤：

1. 选择你要用来训练模型的数据集（如果没有合适的现有数据集，动手收集起来吧）。
2. 创建模型。
3. 训练模型。
4. 在独立的测试集上评估模型，看看它的效果如何。
5. 在训练好的模型中，使用新数据进行预测。

在DeepChem中，每个步骤都可以只用一两行Python代码来完成。在本教程中，我们将通过一个基本的例子来展示如何使用DeepChem完成整个工作流程，以解决一个真实的科学问题。

我们要解决的问题是根据小分子的化学式预测它们的溶解度。这是药物开发中非常重要的一个属性：如果一个候选药物的溶解度不够高，可能无法让足够的药物进入患者的血液中，从而产生治疗效果。首先，我们需要一个包含真实分子溶解度测量数据的数据集。DeepChem的核心组件之一是MoleculeNet，这是一个包含多种化学和分子数据集的集合。在本教程中，我们将使用Delaney溶解度数据集。这个数据集中溶解度的属性以log(溶解度)的形式报告，其中溶解度以摩尔/升为单位进行测量。

In [2]:
# 从MoleculeNet中加载Delaney溶解度数据集，并使用GraphConv作为特征化方法
tasks, datasets, transformers = dc.molnet.load_delaney(featurizer='GraphConv')

# 将加载的数据集分为训练集、验证集和测试集
train_dataset, valid_dataset, test_dataset = datasets

首先，注意传递给`load_delaney()`函数的`featurizer`参数。分子可以用多种方式表示，因此我们需要告诉它我们想使用哪种表示方式，或者用更专业的语言来说，如何对数据进行“特征化”。其次，注意我们实际上得到了三个不同的数据集：训练集、验证集和测试集。在标准的深度学习工作流程中，每个数据集都有不同的功能。

现在我们已经有了数据，下一步是创建一个模型。我们将使用一种叫做“图卷积网络”（graph convolutional network）的模型，简称“graphconv”。

In [ ]:
# 创建一个图卷积网络（GraphConv）模型
# n_tasks=1 指定任务数量为1
# mode='regression' 表示这是一个回归模型，用于预测连续值，例如IC50 = 2.3 nM
# dropout=0.2 设置了0.2的丢弃率，以防止过拟合
model = dc.models.GraphConvModel(n_tasks=1, mode='regression', dropout=0.2)

之后，我们需要在数据集上训练模型。我们只需要将数据集提供给模型，并告诉它要执行多少个训练周期（即完整遍历数据的次数）。


In [ ]:
# 训练模型，使用训练集 train_dataset，训练100个周期
model.fit(train_dataset, nb_epoch=100)  

现在我们已经有了一个完全训练好的模型！但真的是这样吗？为了确认这一点，我们必须在测试集上评估模型。我们可以通过选择一个评估指标并调用 `evaluate()` 函数来实现评估。在这个示例中，我们使用皮尔逊相关系数（也称为r<sup>2</sup>）作为评估指标。我们可以在训练集和测试集上进行评估。


In [5]:
# 定义评估指标，使用皮尔逊相关系数（r²）作为度量
# 在训练集上评估模型，并输出得分
# 在测试集上评估模型，并输出得分
metric = dc.metrics.Metric(dc.metrics.pearson_r2_score) 
print("Training set score:", model.evaluate(train_dataset, [metric], transformers))
print("Test set score:", model.evaluate(test_dataset, [metric], transformers))  

Training set score: {'pearson_r2_score': 0.9120586991816808}
Test set score: {'pearson_r2_score': 0.6745782152564651}


请注意!!! 一般情况下，训练集比测试集表现更好是过拟合的表现。因为模型在训练过程中学到了训练数据的细节和噪声，导致它在训练数据上表现得非常好，但在未见过的测试数据上表现较差。过拟合的模型对训练集过度拟合，缺乏对新数据的泛化能力。

此外，如果测试集表现比训练集好，这通常是不正常的，可能是因为训练过程中数据不够或模型设置不合理，也可能存在数据泄漏等问题。这也是为什么我们需要用独立的测试集来评估模型的原因。

----

我们的模型在测试集上的表现也还是相当不错的。作为对比，如果模型的输出完全是随机的，那么相关性会接近0；如果预测非常准确，相关性则会接近1。我们的模型表现不错，说明它可以用来预测其他分子的属性。

因为这只是个教程，我们没有其他特定想要预测的分子，所以这里我们就使用测试集中的前十个分子来做示例。对于每个分子，我们会输出其化学结构（用SMILES表示）以及预测的对数溶解度。为了方便比较，我们还会输出测试集中对应的实际对数溶解度值。


In [6]:
# 预测测试集中前10个分子的溶解度
solubilities = model.predict_on_batch(test_dataset.X[:10])  

# 遍历测试集的分子ID、模型预测的溶解度以及测试集中的实际溶解度
# 输出预测溶解度、实际溶解度和分子ID
for molecule, solubility, test_solubility in zip(test_dataset.ids, solubilities, test_dataset.y):  
    print(solubility, test_solubility, molecule)  

[-1.4672483] [-1.60114461] c1cc2ccc3cccc4ccc(c1)c2c34
[0.4023751] [0.20848251] Cc1cc(=O)[nH]c(=S)[nH]1
[0.04140282] [-0.01602738] Oc1ccc(cc1)C2(OC(=O)c3ccccc23)c4ccc(O)cc4 
[-1.8386158] [-2.82191713] c1ccc2c(c1)cc3ccc4cccc5ccc2c3c45
[-0.9692923] [-0.52891635] C1=Cc2cccc3cccc1c23
[1.4027135] [1.10168349] CC1CO1
[-0.30959255] [-0.88987406] CCN2c1ccccc1N(C)C(=S)c3cccnc23 
[-1.0358635] [-0.52649706] CC12CCC3C(CCc4cc(O)ccc34)C2CCC1=O
[-1.1721545] [-0.76358725] Cn2cc(c1ccccc1)c(=O)c(c2)c3cccc(c3)C(F)(F)F
[0.25937387] [-0.64020358] ClC(Cl)(Cl)C(NC=O)N1C=CN(C=C1)C(NC=O)C(Cl)(Cl)Cl 


恭喜你，完成第一个教程! 